In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jobs-dataset-from-glassdoor/salary_data_cleaned.csv
/kaggle/input/jobs-dataset-from-glassdoor/glassdoor_jobs.csv
/kaggle/input/jobs-dataset-from-glassdoor/eda_data.csv


**Aim:** To predict maximum salary based on the minimum salary

**References:**

Source of Salary Prediction Dataset used: The Devastor (2022) https://www.kaggle.com/datasets/thedevastator/jobs-dataset-from-glassdoor Date Accessed: 05-12-2022

Tutorial used to build this model: Kaggle (2022) https://www.kaggle.com/learn/intro-to-machine-learning Date Accessed: 05-12-2022

In [2]:
import pandas as pd  
salary_file_path = '../input/jobs-dataset-from-glassdoor/salary_data_cleaned.csv'
salary_data = pd.read_csv(salary_file_path)
salary_data.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')

In [3]:
salary_data = salary_data.dropna(axis=0)

In [4]:
y = salary_data.max_salary
salary_features = ['min_salary']
X = salary_data[salary_features]
X.describe()

,min_salary
count,742.000000
mean,74.068733
std,31.869282
min,10.000000
25%,52.000000
50%,69.500000
75%,91.000000
max,202.000000


In [5]:
X.head()

,min_salary
0,53
1,63
2,80
3,56
4,86


In [6]:
from sklearn.tree import DecisionTreeRegressor

salary_model = DecisionTreeRegressor(random_state=1)

salary_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [7]:
print("Making predictions for the following 5 rows of maximum salaries:")
print(X.head())
print("The predictions are")
print(salary_model.predict(X.head()))

Making predictions for the following 5 rows of maximum salaries:
   min_salary
0          53
1          63
2          80
3          56
4          86
The predictions are
[ 95.16666667 109.1875     132.22222222  97.         141.53333333]


In [8]:
from sklearn.metrics import mean_absolute_error
predicted_salary = salary_model.predict(X)
mean_absolute_error(y, predicted_salary)

7.632434535585795

In [9]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)
salary_model = DecisionTreeRegressor()
salary_model.fit(train_X, train_y)
val_predictions = salary_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

9.570492022911377


In [10]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes,random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [11]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d \t \t Mean Absolute Error: %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5 	 	 Mean Absolute Error: 15
Max leaf nodes: 50 	 	 Mean Absolute Error: 9
Max leaf nodes: 500 	 	 Mean Absolute Error: 9
Max leaf nodes: 5000 	 	 Mean Absolute Error: 9


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
salary_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, salary_preds))

9.511016584272577
